In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os

from keras.layers import BatchNormalization, Activation, Dropout
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D
from keras.layers import Dense, Flatten, Input

from keras.models import Model
from keras.layers import concatenate
from keras import backend as k

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [37]:
class GoogLeNet:
    @staticmethod
    def conv_module(inputs, 
                    filters,
                    kernel_size, 
                    stride, 
                    padding,
                    activation="relu"):
        
        x = Conv2D(filters, kernel_size=kernel_size, strides=stride,
                        padding=padding)(inputs)
        
        x = BatchNormalization(axis=-1)(x)
        
        x = Activation(activation)(x)
        return x
    
    
    @staticmethod
    def incpetion_module(inputs,
                         single_filter,
                         filters1X1_1,
                         filters1X1_2,
                         filters2X2_1,
                         filters2X2_2,
                         max_conv_filter):
        single_conv = Conv2D(single_filter, kernel_size=(1, 1), strides=(1, 1),
                        padding="same", activation="relu")(inputs)
        
        conv_1X1 = GoogLeNet.conv_module(inputs, filters1X1_1, (1, 1), 
                                        (1, 1), "same")
        conv_1X1 = GoogLeNet.conv_module(conv_1X1, filters1X1_2, (3, 3),
                                         (1, 1), "same")
        
        
        conv_3X3 = GoogLeNet.conv_module(inputs, filters2X2_1, (1, 1), 
                                        (1, 1), "same")
        conv_3X3 = GoogLeNet.conv_module(conv_3X3, filters2X2_2, (5, 5),
                                         (1, 1), "same")
        
        max_conv = MaxPooling2D((1, 1), (1, 1))(inputs)
        max_conv = GoogLeNet.conv_module(max_conv, max_conv_filter, (1, 1), (1, 1), "same")
        
        x = concatenate([single_conv, conv_1X1, conv_3X3, max_conv], axis=-1)
        return x
        
    @staticmethod
    def build(width, height, depth, classes):
        inputShape = [height, width, depth]
        
        inputs = Input(shape=inputShape)
        
        x = GoogLeNet.conv_module(inputs, 64, (7, 7), (2, 2), "same")
        x = MaxPooling2D((3, 3), (2, 2))(x)
        
        x = GoogLeNet.conv_module(x, 64, (1, 1), (1, 1), "same")
        x = GoogLeNet.conv_module(x, 192, (3, 3), (1, 1), "same")
        x = MaxPooling2D((3, 3), (2, 2))(x)
        
        x = GoogLeNet.incpetion_module(x, 64, 96, 128, 16, 32, 32)
        x = GoogLeNet.incpetion_module(x, 128, 128, 192, 32, 96, 64)
        x = MaxPooling2D((3, 3), (2, 2))(x)
        
        x = GoogLeNet.incpetion_module(x, 192, 96, 208, 16, 48, 64)
        x = GoogLeNet.incpetion_module(x, 160, 112, 224, 16, 48, 64)
        x = GoogLeNet.incpetion_module(x, 128, 128, 256, 24, 64, 64)
        x = GoogLeNet.incpetion_module(x, 112, 144, 288, 32, 64, 64)
        x = GoogLeNet.incpetion_module(x, 256, 160, 320, 32, 128, 128)
        x = MaxPooling2D((3, 3), (2, 2))(x)
        
        x = GoogLeNet.incpetion_module(x, 256, 160, 320, 32, 128, 128)
        x = GoogLeNet.incpetion_module(x, 384, 192, 384, 48, 128, 128)
        
        x = AveragePooling2D((6, 6), (1, 1))(x)
        x = Dropout(0.4)(x)
        x = Flatten()(x)
        x = Dense(classes, activation="softmax")(x)
        
        model = Model(inputs, x, name="GoogLeNet")
        return model

In [38]:
keras.backend.clear_session()

In [39]:
model = GoogLeNet.build(227, 227, 3, 1000)

In [40]:
model.summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 114, 114, 64) 9472        input_9[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 114, 114, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 114, 114, 64) 0           batch_normalization[0][0]        
__________________________________________________________________________________________